In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns


import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/src/script.ipynb
/kaggle/lib/kaggle/gcp.py
/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-jan-2021/train.csv
/kaggle/input/tabular-playground-series-jan-2021/test.csv
/kaggle/working/__notebook__.ipynb


In [2]:
train_df = pd.read_csv(r'/kaggle/input/tabular-playground-series-jan-2021/train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')

test_id = test_df.id
test = test_df.drop('id', axis=1)

In [3]:
X, y = train_df.iloc[:,1:-1], train_df.iloc[:,-1]

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)


# **XGBoost**

In [5]:
import xgboost as xgb

Best_trial= {'lambda': 0.0030282073258141168, 'alpha': 0.01563845128469084, 'colsample_bytree': 0.5,
             'subsample': 0.7,'n_estimators': 4000, 'learning_rate': 0.01,'max_depth': 15,
             'random_state': 178, 'min_child_weight': 257}

regressor = xgb.XGBRegressor(**Best_trial)

regressor.fit(X_train, y_train, early_stopping_rounds=15, eval_set=[(X_test, y_test)], verbose=1)

[0]	validation_0-rmse:7.36192
Will train until validation_0-rmse hasn't improved in 15 rounds.
[1]	validation_0-rmse:7.28894
[2]	validation_0-rmse:7.21669
[3]	validation_0-rmse:7.14517
[4]	validation_0-rmse:7.07439
[5]	validation_0-rmse:7.00433
[6]	validation_0-rmse:6.93496
[7]	validation_0-rmse:6.86628
[8]	validation_0-rmse:6.79827
[9]	validation_0-rmse:6.73100
[10]	validation_0-rmse:6.66439
[11]	validation_0-rmse:6.59844
[12]	validation_0-rmse:6.53319
[13]	validation_0-rmse:6.46861
[14]	validation_0-rmse:6.40466
[15]	validation_0-rmse:6.34140
[16]	validation_0-rmse:6.27874
[17]	validation_0-rmse:6.21670
[18]	validation_0-rmse:6.15528
[19]	validation_0-rmse:6.09450
[20]	validation_0-rmse:6.03432
[21]	validation_0-rmse:5.97480
[22]	validation_0-rmse:5.91585
[23]	validation_0-rmse:5.85751
[24]	validation_0-rmse:5.79975
[25]	validation_0-rmse:5.74254
[26]	validation_0-rmse:5.68593
[27]	validation_0-rmse:5.62988
[28]	validation_0-rmse:5.57442
[29]	validation_0-rmse:5.51954
[30]	validation

XGBRegressor(alpha=0.01563845128469084, base_score=0.5, booster='gbtree',
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', lambda=0.0030282073258141168,
             learning_rate=0.01, max_delta_step=0, max_depth=15,
             min_child_weight=257, missing=nan, monotone_constraints='()',
             n_estimators=4000, n_jobs=0, num_parallel_tree=1, random_state=178,
             reg_alpha=0.015638452, reg_lambda=0.00302820723,
             scale_pos_weight=1, subsample=0.7, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [6]:
def metric(y_true, y_pred):
    m = np.sum((y_true - y_pred)**2)
    m /= y_pred.shape
    return np.sqrt(m)

In [7]:
metric(y_test, regressor.predict(X_test))

array([0.69246998])

In [8]:
submission = pd.concat([test_id, pd.Series(regressor.predict(test))], axis=1)
submission.columns = ['id', 'target']
submission.to_csv("submission.csv", index=False)